In [1]:
!pip install openeo openeo-pg-parser-networkx openeo[localprocessing] openeo-processes-dask openeo-processes

  Obtaining dependency information for openeo from https://files.pythonhosted.org/packages/e1/d1/5bc45bdbc80e8b3ece04838a32b9e6bdd5f9a87bb4fff7bc9238cd757b1b/openeo-0.29.0-py3-none-any.whl.metadata
  Using cached openeo-0.29.0-py3-none-any.whl.metadata (7.3 kB)
  Obtaining dependency information for openeo-pg-parser-networkx from https://files.pythonhosted.org/packages/5a/68/a46888c84a7acb5124c3e867fe8c35c9cbf593845d30e7ed914d5c769c06/openeo_pg_parser_networkx-2024.5.0-py3-none-any.whl.metadata
  Using cached openeo_pg_parser_networkx-2024.5.0-py3-none-any.whl.metadata (7.2 kB)
  Obtaining dependency information for openeo-processes-dask from https://files.pythonhosted.org/packages/08/ce/7238c5c1e47d1461a02351e29a3c007e410afd31acceb13df378bdd2e619/openeo_processes_dask-2024.5.3-py3-none-any.whl.metadata
  Obtaining dependency information for openeo-processes from https://files.pythonhosted.org/packages/08/a8/3d4c12ab6945c3cd2871f94a0bbd8a222315480ef813c3201304dc7e0d6e/openeo_processes-

In [4]:
from openeo.local import LocalConnection
from pathlib import Path
import xarray as xr



In [12]:
path = '/home/romeokienzler/Downloads/S1A_IW_GRDH_1SDV_20220101T090715_20220101T090740_041265_04E78F_73F0_VH.tif'
p = Path(path)

In [13]:
local_conn = LocalConnection(p.parent)

In [14]:
coll = local_conn.describe_collection(path)

In [16]:
cube_dimensions = coll["cube:dimensions"]
cube_dimensions

{'x': {'type': 'spatial',
  'axis': 'x',
  'extent': [-53.128447512491896, -50.52225521020434],
  'reference_system': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'},
 'y': {'type': 'spatial',
  'axis': 'y',
  'extent': [-6.307796460110961, -4.29404308770023],
  'reference_system': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'},
 'bands': {'type': 'bands', 'values': ['band_1']}}

In [21]:

for dim_name, dim_info in cube_dimensions.items():
    if dim_info.get("axis") is not None:
        if dim_info.get("axis") == 'x': 
            west = dim_info["extent"][0]
            east = dim_info["extent"][1]
        if dim_info.get("axis") == 'y': 
            south = dim_info["extent"][0]
            north = dim_info["extent"][1]
coords = list()
for lat in [south, north]:
    for lon in [west, east]:
        coords.append([lon, lat])
datacube = local_conn.load_collection(path)
data: xr.DataArray = datacube.execute()

ValidationError: 34 validation errors for ProcessGraph
process_graph.root.arguments.spatial_extent.ResultReference
  Input should be a valid dictionary or instance of ResultReference [type=model_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.7/v/model_type
process_graph.root.arguments.spatial_extent.ParameterReference
  Input should be a valid dictionary or instance of ParameterReference [type=model_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.7/v/model_type
process_graph.root.arguments.spatial_extent.ProcessGraph
  Input should be a valid dictionary or instance of ProcessGraph [type=model_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.7/v/model_type
process_graph.root.arguments.spatial_extent.BoundingBox
  Input should be a valid dictionary or instance of BoundingBox [type=model_type, input_value=Non

In [ ]:
from openeo.local import LocalConnection
from pathlib import Path
import pytest
import xarray as xr

from tensorlakehouse_openeo_driver.constants import DEFAULT_X_DIMENSION, DEFAULT_Y_DIMENSION

# from tensorlakehouse_openeo_driver.local.connection import GeodnLocalConnection


@pytest.mark.parametrize(
    "path",
    [
        "/Users/ltizzei/Projects/Orgs/IBM/tensorlakehouse-openeo-driver/tensorlakehouse_openeo_driver/tests/test_data/test_openeo_Globalweather-ERA5-.nc",
        "/Users/ltizzei/Projects/Orgs/IBM/tensorlakehouse-openeo-driver/tensorlakehouse_openeo_driver/tests/test_data/tasmax_rcp85_land-cpm_uk_2.2km/tasmax_rcp85_land-cpm_uk_2.2km_01_day_20701201-20711130_reproj.nc",
    ],
)
def test_local_connection(path: str):
    p = Path(path)
    assert p.exists(), f"Error! File does not exist {p}"

    local_conn = LocalConnection(p.parent)

    # list_collections = local_conn.list_collections()
    coll = local_conn.describe_collection(path)
    cube_dimensions = coll["cube:dimensions"]
    temporal_dim_name = None
    for dim_name, dim_info in cube_dimensions.items():
        if dim_info.get("axis") is not None:
            if dim_info.get("axis") == DEFAULT_X_DIMENSION: 
                west = dim_info["extent"][0]
                east = dim_info["extent"][1]
            if dim_info.get("axis") == DEFAULT_Y_DIMENSION: 
                south = dim_info["extent"][0]
                north = dim_info["extent"][1]
        elif dim_info.get("type") == "temporal":
            temporal_dim_name = dim_name
    coords = list()
    for lat in [south, north]:
        for lon in [west, east]:
            coords.append([lon, lat])
    datacube = local_conn.load_collection(path)
    datacube = datacube.reduce_dimension(dimension=temporal_dim_name, reducer="mean")
    data: xr.DataArray = datacube.execute()
    assert isinstance(data, xr.DataArray)
    assert temporal_dim_name not in data.dims